# Creazione modello text analysis
Ho visto che il decision tree è il modello che mi offre migliori performance, soprattutto con TfIdf e tokenizzazione nltk/regex con stemming PorterStemmer().


In [19]:
from my_tokenizer import MyTokenizer
#from classification import Classification
#from model import Model
import pandas as pd
import util_strings as utils

È necessaria la normalizzazione delle feature, dato che nel testo abbiamo i nomi delle squadre ed i soprannomi, vado a sostituire tutte le parole relative alla squadra di casa con home team e stessa cosa per la away team.
Questo viene fatto tramite il metodo feature_normalization()

In [20]:
mt = MyTokenizer(pd.read_csv(utils.completed_dataset, index_col=0))
#mt.feature_normalization()
mt.clean_text()
X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets()
mt.save_vectorizer(utils.vectorizer)

## Classificazione del modello
Migliori risultati con Decision Tree.
Viene salvato il modello

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model = DecisionTreeClassifier()

In [15]:
import sklearn.metrics as mtr
E = []

accuracy = 0
while(accuracy < 0.94):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    analysis = pd.DataFrame()
            
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    if report['accuracy'] > accuracy: 
        accuracy = report['accuracy']
        print(accuracy)

0.9338842975206612
0.9380165289256198
0.9421487603305785


In [16]:
analysis = {
    'Model': 'Model', 'Accuracy': report['accuracy'],
    'Avg Precision (macro)': report['macro avg']['precision'],
    'Avg Recall (macro)': report['macro avg']['recall'],
    'Avg F1-score (macro)': report['macro avg']['f1-score'],
    'Avg Precision (weighted)': report['weighted avg']['precision'],
    'Avg Recall (weighted)': report['weighted avg']['recall'],
    'Avg F1-score (weighted)': report['weighted avg']['f1-score']
}

In [17]:
import pickle

In [18]:
with open(utils.text_analysis, 'wb') as file:
    pickle.dump(model, file)